<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук ВШЭ, программист-исследователь Mail.ru Group  Юрий Кашницкий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 9. Разреженные данные, онлайн-обучение</center>
## <center>Часть 4. Классификация отзывов к фильмам с SVM и логистической регрессией</center>

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%pylab inline
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

Populating the interactive namespace from numpy and matplotlib


**Загрузим данные [отсюда](https://yadi.sk/d/Tg1Tflur333iLr). В обучающей и тестовой выборках по 12500 тысяч хороших и плохих отзывов к фильмам.**

In [2]:
# поменяйте путь к файлу
reviews_train = load_files("C:/Users/Ksenia/OneDrive/ML_HSE_BIG/module3_advanced_ML/data/aclImdb/test/")
text_train, y_train = reviews_train.data, reviews_train.target

In [3]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))

Number of documents in training data: 17239
[12500  4739]


In [4]:
# поменяйте путь к файлу
reviews_test = load_files("/Users/yorko/Яндекс.Диск/ML/data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '/Users/yorko/Яндекс.Диск/ML/data/aclImdb/test/'

**Пример отзыва и соответствующей метки.**

In [15]:
text_train[4]

b'When one of my friends recommended this to me, raving about how well it was filmed, the underlying themes and the general greatness of the film, I obviously expected an amazing, at least entertaining film.<br /><br />The two hours I spent watching this turned out to be a huge disappointing waste of my time.<br /><br />I understand that this movie is meant to be surreal, but even in surreal movies, there is something which anchors it down, even if it is only in the slightest. This movie, on the other hand, felt forced and fake. A lot of the shots were unnecessary and watching it made me think the director was trying to hard to be artistic.<br /><br />The acting was poor, and the relationships between characters were not nearly developed enough. Maybe that\'s just me missing something that others could see but I hadn\'t even realised there was any sort of attraction between Dae su and Migo before they started getting at it like rabbits randomly half way through the film. Then again, ma

In [17]:
y_train[4] # плохой отзыв

0

In [13]:
text_train[3]

b'Beautiful images, propaganda and history as toy. The ingredients of this film, good, interesting but with many shadows. Politicall shadows. Jefferson is more than the hero of a mythical America. He is silhouette of a vision about life and society, an extraordinary thinker, teacher of large part of North America. In film is only Superman. The delicate contour of ideas are crushed. The subtle birth of essential truth is forgotten. And Jefferson is basic instrument for create a good image of American realities. The end,triumphal and fake, romance and heroic, is gun suicide of a story who may be tall with more wisdom. And the war against "Avatar" is another gray stain. For one who heard nothing about Jefferson, movie is a good beginning. For the others - disillusion. It is like the precious silk of a tailor. But the tailor is fascinating about silk and the clothes are only ordinary cloth. So, a sad experience. Slices of beauty and boring lakes, patriotic lesson and the crush of expectati

In [14]:
y_train[3] # хороший отзыв

0

**Идея "мешка слов"**

<img src="../../img/bag_of_words.svg" width=80%>

## Простой подсчет слов

**Составим словарь всех слов с помощью CountVectorizer.**

In [ ]:
cv = CountVectorizer()
cv.fit(text_train)

len(cv.vocabulary_)

**Посмотрим на примеры полученных "слов" (лучше их называть токенами). Видим, что многие важные этапы обработки текста мы тут пропустили.**

In [ ]:
print(cv.get_feature_names()[:50])
print(cv.get_feature_names()[50000:50050])

**Закодируем предложения из текстов обучающей выборки индексами входящих слов. Используем разреженный формат.**

In [ ]:
X_train = cv.transform(text_train)
X_train

**Посмотрим, как преобразование подействовало на одно из предложений.**

In [ ]:
print(text_train[19726])

In [ ]:
X_train[19726].nonzero()[1]

In [ ]:
X_train[19726].nonzero()

**Преобразуем так же тестовую выборку.**

In [ ]:
X_test = cv.transform(text_test)

**Обучим логистическую регрессию и линейный SVM.**

In [ ]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=7)
logit.fit(X_train, y_train)

In [ ]:
%%time
svm = LinearSVC(random_state=7)
svm.fit(X_train, y_train)

**Посмотрим на доли правильных ответов на обучающей и тестовой выборках.**

In [ ]:
round(logit.score(X_train, y_train), 3), round(svm.score(X_train, y_train), 3)

In [ ]:
round(logit.score(X_test, y_test), 3), round(svm.score(X_test, y_test), 3)

In [ ]:
def visualize_coefficients(classifier, feature_names, n_top_features=25):
    # get coefficients with large absolute values 
    coef = classifier.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_top_features:]
    negative_coefficients = np.argsort(coef)[:n_top_features]
    interesting_coefficients = np.hstack([negative_coefficients, positive_coefficients])
    # plot them
    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[interesting_coefficients]]
    plt.bar(np.arange(2 * n_top_features), coef[interesting_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1 + 2 * n_top_features), feature_names[interesting_coefficients], rotation=60, ha="right");


In [ ]:
def plot_grid_scores(grid, param_name):
    plot(grid.param_grid[param_name], grid.cv_results_['mean_train_score'],
        color='green', label='train')
    plot(grid.param_grid[param_name], grid.cv_results_['mean_test_score'],
        color='red', label='test')
    legend();
    

In [ ]:
visualize_coefficients(logit, cv.get_feature_names())

In [ ]:
visualize_coefficients(svm, cv.get_feature_names())

**Подберем коэффициент регуляризации для логистической регрессии.**

In [ ]:
%%time
from sklearn.pipeline import make_pipeline

text_pipe_logit = make_pipeline(CountVectorizer(), LogisticRegression(n_jobs=-1, random_state=7))

text_pipe_logit.fit(text_train, y_train)
print(text_pipe_logit.score(text_test, y_test))

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid_logit = {'logisticregression__C': np.logspace(-5, 0, 6)}
grid_logit = GridSearchCV(text_pipe_logit, param_grid_logit, cv=3, n_jobs=-1)

grid_logit.fit(text_train, y_train)

**Лучшее значение C и соответствующее качество на кросс-валидации.**

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

In [ ]:
plot_grid_scores(grid_logit, 'logisticregression__C')

**То же самое для LinearSVC.**

In [ ]:
%%time
text_pipe_svm = make_pipeline(CountVectorizer(), LinearSVC(random_state=7))

text_pipe_svm.fit(text_train, y_train)
print(text_pipe_svm.score(text_test, y_test))

In [ ]:
%%time
param_grid_svm = {'linearsvc__C': np.logspace(-5, 0, 6)}
grid_svm = GridSearchCV(text_pipe_svm, param_grid_svm, cv=3, n_jobs=-1)

grid_svm.fit(text_train, y_train);

In [ ]:
grid_svm.best_params_, grid_svm.best_score_

In [ ]:
plot_grid_scores(grid_svm, 'linearsvc__C')

In [ ]:
visualize_coefficients(grid_svm.best_estimator_.named_steps['linearsvc'],
                       grid_svm.best_estimator_.named_steps['countvectorizer'].get_feature_names())

На валидационной выборке:

In [ ]:
grid_logit.score(text_test, y_test), grid_svm.score(text_test, y_test)

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_pipe = make_pipeline(TfidfVectorizer(), LinearSVC())

param_grid = {'linearsvc__C': np.logspace(-3, 2, 6)}
grid_tfidf = GridSearchCV(tfidf_pipe, param_grid, cv=3, n_jobs=-1)
grid_tfidf.fit(text_train, y_train)
plot_grid_scores(grid_tfidf, 'linearsvc__C')

In [ ]:
visualize_coefficients(grid_tfidf.best_estimator_.named_steps['linearsvc'],
                       grid_tfidf.best_estimator_.named_steps['tfidfvectorizer'].get_feature_names())

In [ ]:
grid_tfidf.best_score_, grid_tfidf.best_params_

## N-граммы

In [ ]:
%%time
text_pipe = make_pipeline(CountVectorizer(), LinearSVC())

param_grid = {'linearsvc__C': [0.01, 0.1, 1],
              "countvectorizer__ngram_range": [(1, 2)]}

grid_bigram = GridSearchCV(text_pipe, param_grid, cv=3)

grid_bigram.fit(text_train, y_train)

In [ ]:
plot_grid_scores(grid_bigram, 'linearsvc__C')

In [ ]:
grid_bigram.best_score_